## Project

----

## Send Data over Audio

In [ ]:
# Import libraries
import ggwave
import subprocess
import tempfile
import os

# Define the send function
def send_message(message="hello"):
    """
    Send a message using ggwave
    """
    print(f"Sending message: '{message}'")
    
    # Initialize ggwave
    instance = ggwave.init()
    
    try:
        # Encode the message
        waveform = ggwave.encode(message)
        
        # Create a temporary file
        with tempfile.NamedTemporaryFile(suffix='.raw', delete=False) as temp_file:
            temp_filename = temp_file.name
            temp_file.write(waveform)
            temp_file.flush()
        
        # Play with SoX
        print("Playing audio...")
        subprocess.run([
            'play', 
            '-t', 'f32', 
            '-r', '48000',  
            '-c', '1',      
            '-b', '32',     
            temp_filename
        ], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        
        print("Transmission complete")
        
        # Clean up
        os.unlink(temp_filename)
        
    except Exception as e:
        print(f"Error: {e}")
    
    finally:
        # Clean up ggwave
        ggwave.free(instance)

send_message("Brayden Hand")

## Receive Data over Audio (Listener)

In [ ]:
import ggwave
import subprocess
import tempfile
import os
import time

def listen_for_messages(duration=60):
    """
    Listen for GGWave messages for the specified duration (in seconds)
    """
    print(f"Listening for {duration} seconds...")
    
    # Initialize ggwave
    instance = ggwave.init()
    
    # To keep track of already received messages
    received_messages = set()
    
    try:
        # Create a temporary file for SoX to write to
        with tempfile.NamedTemporaryFile(suffix='.raw', delete=False) as temp_file:
            temp_filename = temp_file.name
        
        # Start recording in the background
        sox_process = subprocess.Popen([
            'rec',
            '-q',                # Quiet mode
            '-t', 'f32',         # Same format as sender
            '-r', '48000',       # Same sample rate as sender
            '-c', '1',           # Mono
            '-b', '32',          # 32-bit
            temp_filename
        ], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        
        # Wait and check for messages
        start_time = time.time()
        end_time = start_time + duration
        last_check_size = 0
        
        while time.time() < end_time:
            # Sleep to allow recording to accumulate
            time.sleep(1)
            
            try:
                # Get the current size of the recording file
                current_size = os.path.getsize(temp_filename)
                
                # Only process if there's new data
                if current_size > last_check_size:
                    # Read the current file content
                    with open(temp_filename, 'rb') as f:
                        # Only read the new data
                        f.seek(last_check_size)
                        new_data = f.read()
                    
                    # Update the last checked size
                    last_check_size = current_size
                    
                    # Try to decode
                    result = ggwave.decode(instance, new_data)
                    
                    # If we got a message, print it (but only if new)
                    if result is not None:
                        message = result.decode('utf-8')
                        
                        # Only print if we haven't seen this message before
                        if message not in received_messages:
                            print(f"Received: \"{message}\"")
                            received_messages.add(message)
            except Exception as e:
                # Quietly handle errors
                pass
                
    except KeyboardInterrupt:
        print("\nStopped listening.")
    finally:
        # Clean up
        if 'sox_process' in locals():
            sox_process.terminate()
        
        if 'temp_filename' in locals() and os.path.exists(temp_filename):
            os.unlink(temp_filename)
        
        ggwave.free(instance)
        
        if not received_messages:
            print("No messages received.")
        else:
            print(f"Listening complete. Received {len(received_messages)} unique messages.")

# Start listening for 60 seconds
listen_for_messages(60)

------

## AutoEncoder

In [3]:
import requests
import os
import time
import tempfile
import subprocess
import re
import collections
import pickle
import ggwave  # Add explicit import for ggwave

# Add TextCompressor class definition
class TextCompressor:
    """
    A text compression system that uses frequency analysis of words, syllables, 
    and character patterns to create an optimized encoding dictionary.
    """
    
    def __init__(self, load_path=None):
        """Initialize the compressor with an optional pre-trained dictionary"""
        if load_path and os.path.exists(load_path):
            self.load_dictionary(load_path)
        else:
            # Dictionary mapping patterns to their encoded representations
            self.encoding_dict = {}
            # Reverse mapping for decoding
            self.decoding_dict = {}
            # Reserve first 32 characters (non-printable ASCII) as encoding markers
            self.next_code = 1
            # Special marker to identify compressed content
            self.COMPRESSION_MARKER = chr(0)
    
    def train(self, corpus, min_frequency=5, max_patterns=256):
        """
        Train the compressor on a text corpus to identify common patterns
        
        Args:
            corpus: Text to analyze for patterns
            min_frequency: Minimum frequency for a pattern to be included
            max_patterns: Maximum number of patterns to encode
        """
        # Tokenize the corpus into words and analyze frequency
        words = re.findall(r'\w+', corpus.lower())
        word_freq = collections.Counter(words)
        
        # Find common character sequences (n-grams)
        ngrams = []
        for n in range(2, 5):  # 2-grams, 3-grams, 4-grams
            for i in range(len(corpus) - n + 1):
                ngrams.append(corpus[i:i+n])
        ngram_freq = collections.Counter(ngrams)
        
        # Combine word and ngram frequencies
        combined_freq = {}
        for item, freq in word_freq.items():
            if len(item) > 2 and freq >= min_frequency:  # Only include words longer than 2 chars
                # Calculate space saving: frequency * (len + 1 for space - 1 for code)
                saving = freq * len(item)
                combined_freq[item] = (freq, saving)
                
        for item, freq in ngram_freq.items():
            if freq >= min_frequency and item not in combined_freq:
                saving = freq * (len(item) - 1)
                combined_freq[item] = (freq, saving)
        
        # Sort by space saving potential
        sorted_patterns = sorted(combined_freq.items(), key=lambda x: x[1][1], reverse=True)
        
        # Take top patterns, limited by max_patterns
        top_patterns = sorted_patterns[:max_patterns]
        
        # Create encoding dictionary using control characters (1-31) and extended ASCII
        self.encoding_dict = {}
        self.decoding_dict = {}
        
        for i, (pattern, _) in enumerate(top_patterns):
            # Start at 1 to avoid NULL character (0), use extended ASCII after 31
            code = chr(1 + i) if i < 31 else chr(128 + (i - 31))
            self.encoding_dict[pattern] = code
            self.decoding_dict[code] = pattern
        
        # Always include the compression marker
        self.COMPRESSION_MARKER = chr(0)
        
        return self.encoding_dict
    
    def encode(self, text):
        """
        Compress text using the trained dictionary
        
        Args:
            text: The text to compress
            
        Returns:
            Compressed text with the compression marker prefix
        """
        if not self.encoding_dict:
            return text  # Can't compress without a dictionary
        
        compressed = text
        
        # Sort patterns by length (longest first) to avoid partial matches
        patterns = sorted(self.encoding_dict.keys(), key=len, reverse=True)
        
        # Replace each pattern with its code
        for pattern in patterns:
            code = self.encoding_dict[pattern]
            # For words, ensure we're replacing whole words
            if re.match(r'^\w+$', pattern):
                compressed = re.sub(r'\b' + re.escape(pattern) + r'\b', code, compressed, flags=re.IGNORECASE)
            else:
                compressed = compressed.replace(pattern, code)
        
        # Add compression marker to indicate this is compressed
        return self.COMPRESSION_MARKER + compressed
    
    def decode(self, compressed_text):
        """
        Decompress text using the dictionary
        
        Args:
            compressed_text: Compressed text starting with the compression marker
            
        Returns:
            Decompressed text or original if not compressed
        """
        # Check if this is actually compressed
        if not compressed_text or compressed_text[0] != self.COMPRESSION_MARKER:
            return compressed_text
        
        # Remove the compression marker
        text = compressed_text[1:]
        
        # Replace each code with its pattern
        for code, pattern in self.decoding_dict.items():
            text = text.replace(code, pattern)
        
        return text
    
    def save_dictionary(self, path):
        """Save the encoding dictionary to a file"""
        data = {
            'encoding_dict': self.encoding_dict,
            'decoding_dict': self.decoding_dict,
            'compression_marker': ord(self.COMPRESSION_MARKER)
        }
        with open(path, 'wb') as f:
            pickle.dump(data, f)
    
    def load_dictionary(self, path):
        """Load the encoding dictionary from a file"""
        with open(path, 'rb') as f:
            data = pickle.load(f)
        
        self.encoding_dict = data['encoding_dict']
        self.decoding_dict = data['decoding_dict']
        self.COMPRESSION_MARKER = chr(data['compression_marker'])
    
    def compression_stats(self, original, compressed):
        """Calculate compression statistics"""
        original_size = len(original.encode('utf-8'))
        compressed_size = len(compressed.encode('utf-8'))
        ratio = original_size / compressed_size if compressed_size > 0 else 0
        
        return {
            'original_size': original_size,
            'compressed_size': compressed_size,
            'compression_ratio': ratio,
            'space_saving': (1 - 1/ratio) * 100 if ratio > 0 else 0
        }

def create_deep_corpus(output_file="deep_corpus.txt", corpus_size=10000000):
    """
    Create a deep English corpus by downloading classic literature texts
    from Project Gutenberg. This creates a rich training set for compression.
    
    Args:
        output_file: File to save the corpus to
        corpus_size: Approximate size of corpus in characters
    
    Returns:
        Path to the created corpus file
    """
    
    # List of classic literature texts from Project Gutenberg (public domain)
    gutenberg_texts = [
    # Classic Literature - Fiction
    "https://www.gutenberg.org/files/1342/1342-0.txt",  # Pride and Prejudice by Jane Austen
    "https://www.gutenberg.org/files/84/84-0.txt",      # Frankenstein by Mary Shelley
    "https://www.gutenberg.org/files/98/98-0.txt",      # A Tale of Two Cities by Charles Dickens
    "https://www.gutenberg.org/files/2701/2701-0.txt",  # Moby Dick by Herman Melville
    "https://www.gutenberg.org/files/345/345-0.txt",    # Dracula by Bram Stoker
    "https://www.gutenberg.org/files/76/76-0.txt",      # Adventures of Huckleberry Finn by Mark Twain
    "https://www.gutenberg.org/files/1661/1661-0.txt",  # Sherlock Holmes by Arthur Conan Doyle
    "https://www.gutenberg.org/files/174/174-0.txt",    # The Picture of Dorian Gray by Oscar Wilde
    "https://www.gutenberg.org/files/145/145-0.txt",    # Middlemarch by George Eliot
    
    # Classic Literature - Additional Genres
    "https://www.gutenberg.org/files/2641/2641-0.txt",  # A Room with a View by E.M. Forster
    "https://www.gutenberg.org/files/11/11-0.txt",      # Alice's Adventures in Wonderland by Lewis Carroll
    "https://www.gutenberg.org/files/1400/1400-0.txt",  # Great Expectations by Charles Dickens
    "https://www.gutenberg.org/files/768/768-0.txt",    # Wuthering Heights by Emily Brontë
    "https://www.gutenberg.org/files/2814/2814-0.txt",  # Dubliners by James Joyce
    ]
    
    # Create corpus by downloading and concatenating texts
    corpus_text = ""
    texts_used = 0
    
    for url in gutenberg_texts:
        try:
            print(f"Downloading {url}")
            response = requests.get(url, timeout=10)
            if response.status_code == 200:
                # Extract the text, removing Project Gutenberg header/footer
                text = response.text
                
                # Remove Gutenberg header (everything before the first empty line after "START")
                start_marker = "*** START OF"
                if start_marker in text:
                    start_pos = text.find(start_marker)
                    text = text[start_pos:]
                    # Find the first paragraph break after the marker
                    para_break = text.find("\r\n\r\n")
                    if para_break > 0:
                        text = text[para_break+4:]
                
                # Remove Gutenberg footer (everything after "END")
                end_marker = "*** END OF"
                if end_marker in text:
                    end_pos = text.find(end_marker)
                    text = text[:end_pos]
                
                # Add to corpus
                corpus_text += text + "\n\n"
                texts_used += 1
                
                # Check if we've reached the target size
                if len(corpus_text) >= corpus_size:
                    break
            
        except Exception as e:
            print(f"Error downloading {url}: {e}")
    
    # Save corpus to file
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(corpus_text)
    
    print(f"Created corpus with {len(corpus_text)} characters from {texts_used} texts")
    print(f"Saved to {output_file}")
    
    return output_file

def train_deep_compressor(corpus_path=None, output_path="deep_compressor.pkl", max_patterns=512):
    """
    Train a TextCompressor on a deep English corpus
    
    Args:
        corpus_path: Path to corpus file (will create one if None)
        output_path: Where to save the trained compressor
        max_patterns: Maximum number of patterns to include in the dictionary
        
    Returns:
        Trained TextCompressor instance
    """
    # Create corpus if needed
    if not corpus_path:
        corpus_path = create_deep_corpus()
    
    print(f"Training compressor on {corpus_path} with max_patterns={max_patterns}")
    
    # Load corpus
    with open(corpus_path, 'r', encoding='utf-8') as f:
        corpus = f.read()
    
    # Create and train compressor
    compressor = TextCompressor()
    
    # Train with a lower minimum frequency for more patterns but higher max_patterns
    start_time = time.time()
    compressor.train(corpus, min_frequency=3, max_patterns=max_patterns)
    training_time = time.time() - start_time
    
    print(f"Training completed in {training_time:.1f} seconds")
    print(f"Trained dictionary has {len(compressor.encoding_dict)} patterns")
    
    # Save the trained compressor
    compressor.save_dictionary(output_path)
    print(f"Saved compressor to {output_path}")
    
    return compressor

def send_encoded_message(message, compressor=None, protocol_id=6, max_bytes=125):
    """
    Compress and send a message using ggwave with byte-size-aware chunking
    
    Args:
        message: Text message to compress and send
        compressor: Optional TextCompressor instance (will create or load one if None)
        protocol_id: GGWave protocol ID (higher = better for longer messages)
        max_bytes: Maximum size for a single transmission chunk in bytes
    """
    # Use provided compressor or load a pre-trained one
    if compressor is None:
        if os.path.exists("deep_compressor.pkl"):
            compressor = TextCompressor("deep_compressor.pkl")
        else:
            compressor = train_deep_compressor()
    
    # Initial character-based chunk size estimate (start conservative)
    # We'll adjust this based on actual compressed size
    estimated_chars_per_chunk = 50  # Start with a conservative estimate
    
    # Split message into chunks, ensuring each compressed chunk is under max_bytes
    chunks = []
    start = 0
    
    while start < len(message):
        # Start with the estimated chunk size
        end = min(start + estimated_chars_per_chunk, len(message))
        
        # Get potential chunk
        potential_chunk = message[start:end]
        
        # Test compress it
        chunk_header = f"[0/0] "  # Temporary header
        test_chunk = chunk_header + potential_chunk
        compressed = compressor.encode(test_chunk)
        
        # Check compressed size in bytes
        compressed_size = len(compressed.encode('utf-8'))
        
        # Binary search to find optimal chunk size
        # If too big, reduce; if small enough, try adding more
        min_size = 1
        max_size = end - start
        
        while min_size < max_size:
            if compressed_size > max_bytes:
                # Too big, reduce size
                max_size = (min_size + max_size) // 2
                potential_chunk = message[start:start + max_size]
                test_chunk = chunk_header + potential_chunk
                compressed = compressor.encode(test_chunk)
                compressed_size = len(compressed.encode('utf-8'))
            else:
                # Small enough, try increasing
                old_min = min_size
                min_size = min_size + (max_size - min_size) // 2
                if min_size == old_min:
                    min_size = max_size  # Break if no progress
                
                if start + min_size >= len(message):
                    # Reached end of message
                    potential_chunk = message[start:]
                    break
                
                larger_chunk = message[start:start + min_size]
                test_chunk = chunk_header + larger_chunk
                compressed = compressor.encode(test_chunk)
                compressed_size = len(compressed.encode('utf-8'))
                
                if compressed_size <= max_bytes:
                    potential_chunk = larger_chunk  # Update if still under limit
        
        # Add this chunk and move to next section
        chunks.append(potential_chunk)
        start += len(potential_chunk)
        
        # Update the estimate for next iteration based on compressed ratio
        content_bytes = len(potential_chunk.encode('utf-8'))
        if content_bytes > 0:
            # Calculate actual compressed ratio for this chunk
            compressed_ratio = compressed_size / content_bytes
            # Estimate for next chunk
            estimated_chars_per_chunk = int((max_bytes / compressed_ratio) * 0.9)  # 10% safety margin
            # Keep it in a reasonable range
            estimated_chars_per_chunk = max(10, min(estimated_chars_per_chunk, 100))
    
    print(f"Message split into {len(chunks)} chunks based on byte size limit of {max_bytes} bytes")
    
    # Send each chunk with properly numbered headers
    for i, chunk in enumerate(chunks):
        chunk_header = f"[{i+1}/{len(chunks)}] "
        full_chunk = chunk_header + chunk
        
        # Double-check size before sending
        compressed = compressor.encode(full_chunk)
        compressed_size = len(compressed.encode('utf-8'))
        
        print(f"\nSending chunk {i+1}/{len(chunks)} ({compressed_size} bytes):")
        
        if compressed_size > max_bytes:
            print(f"WARNING: Chunk size {compressed_size} bytes exceeds target {max_bytes} bytes")
            print("This might cause transmission issues")
        
        # Send this chunk
        _send_single_chunk(full_chunk, compressor, protocol_id)
        
        # Wait between chunks to avoid overlap
        time.sleep(2)
    
    return chunks

def _send_single_chunk(message, compressor, protocol_id=6):
    """Helper function to compress and send a single message chunk"""
    # Compress the message
    original_message = message
    compressed_message = compressor.encode(message)
    
    # Print only the required information
    stats = compressor.compression_stats(original_message, compressed_message)
    print(f"Original message: '{original_message}'")
    print(f"Compressed message: {repr(compressed_message)}")
    print(f"Original: {stats['original_size']} bytes, Compressed: {stats['compressed_size']} bytes")
    print(f"Compression ratio: {stats['compression_ratio']:.2f}x")
    
    try:
        # Initialize ggwave
        instance = ggwave.init()
        
        # Encode the message with ggwave using the specified protocol
        # protocol_id controls type of transmission (higher values usually support longer messages)
        waveform = ggwave.encode(compressed_message, instance, protocol_id)
        
        # Create a temporary file
        with tempfile.NamedTemporaryFile(suffix='.raw', delete=False) as temp_file:
            temp_filename = temp_file.name
            temp_file.write(waveform)
            temp_file.flush()
        
        # Play the audio without additional output
        print("Transmitting...")
        subprocess.run([
            'play',
            '-t', 'f32',
            '-r', '48000',
            '-c', '1',
            '-b', '32',
            temp_filename
        ], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        
        print("Transmission complete")
        
        # Clean up
        os.unlink(temp_filename)
        
        # Free ggwave
        ggwave.free(instance)
        
    except Exception as e:
        print(f"Error: {e}")
    
    return compressed_message

def listen_for_deep_encoded_messages(duration=60, protocol_id=6):
    """
    Listen for deeply encoded messages using the trained compressor
    
    Args:
        duration: How long to listen in seconds
        protocol_id: GGWave protocol ID to listen for (should match sender)
    """
    # Load the deep compressor if available
    if os.path.exists("deep_compressor.pkl"):
        compressor = TextCompressor("deep_compressor.pkl")
    else:
        compressor = train_deep_compressor()
    
    # Define a simple listen function
    def listen_for_encoded_messages(duration, compressor):
        """Simple implementation of the listen function with support for chunked messages"""
        print(f"Listening for encoded messages for {duration} seconds...")
        
        # Initialize ggwave
        instance = ggwave.init()
        
        # Track received messages and message chunks
        received_messages = set()
        chunk_buffer = {}  # Format: {msg_id: {total_chunks: n, chunks: {1: "text", 2: "more text"}}}
        
        try:
            # Create a temporary file for recording
            with tempfile.NamedTemporaryFile(suffix='.raw', delete=False) as temp_file:
                temp_filename = temp_file.name
            
            # Start recording
            sox_process = subprocess.Popen([
                'rec',
                '-t', 'f32',
                '-r', '48000',
                '-c', '1',
                '-b', '32',
                temp_filename
            ], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
            
            # Wait and check for messages
            start_time = time.time()
            end_time = start_time + duration
            last_check_size = 0
            
            while time.time() < end_time:
                time.sleep(1)
                
                try:
                    # Check for new data
                    current_size = os.path.getsize(temp_filename)
                    
                    if current_size > last_check_size:
                        # Read new data
                        with open(temp_filename, 'rb') as f:
                            f.seek(last_check_size)
                            new_data = f.read()
                        
                        last_check_size = current_size
                        
                        # Try to decode with the specified protocol
                        result = ggwave.decode(instance, new_data, protocol_id)
                        
                        if result is not None:
                            message = result.decode('utf-8')
                            
                            # Check if compressed
                            is_compressed = message and message[0] == compressor.COMPRESSION_MARKER
                            
                            if is_compressed:
                                original = message
                                decoded_message = compressor.decode(message)
                                stats = compressor.compression_stats(decoded_message, original)
                                
                                # Check if this is a chunked message
                                chunk_match = re.match(r'\[(\d+)/(\d+)\] (.*)', decoded_message)
                                
                                if chunk_match:
                                    # This is a chunked message
                                    chunk_num = int(chunk_match.group(1))
                                    total_chunks = int(chunk_match.group(2))
                                    chunk_content = chunk_match.group(3)
                                    
                                    # Create a unique ID for this message based on total chunks 
                                    # (not perfect but works for demo)
                                    msg_id = f"chunked_msg_{total_chunks}_{time.time():.0f}"
                                    
                                    # Store or update chunk info
                                    if msg_id not in chunk_buffer:
                                        chunk_buffer[msg_id] = {"total_chunks": total_chunks, "chunks": {}}
                                    
                                    # Add this chunk
                                    chunk_buffer[msg_id]["chunks"][chunk_num] = chunk_content
                                    
                                    print(f"Received chunk {chunk_num}/{total_chunks}")
                                    
                                    # Check if we have all chunks for this message
                                    if len(chunk_buffer[msg_id]["chunks"]) == total_chunks:
                                        # Reconstruct the full message
                                        full_message = ""
                                        for i in range(1, total_chunks + 1):
                                            full_message += chunk_buffer[msg_id]["chunks"][i]
                                        
                                        # Add to received messages
                                        if full_message not in received_messages:
                                            print(f"\nReassembled chunked message:")
                                            print(f"Original message: '{full_message}'")
                                            print(f"Received in {total_chunks} chunks")
                                            received_messages.add(full_message)
                                else:
                                    # Regular (non-chunked) message
                                    if decoded_message not in received_messages:
                                        print(f"Original message: '{decoded_message}'")
                                        print(f"Compressed message: {repr(original)}")
                                        print(f"Original: {stats['original_size']} bytes, Compressed: {stats['compressed_size']} bytes")
                                        print(f"Compression ratio: {stats['compression_ratio']:.2f}x")
                                        print()
                                        received_messages.add(decoded_message)
                            else:
                                # Uncompressed message
                                if message not in received_messages:
                                    print(f"Received uncompressed message: {message}")
                                    received_messages.add(message)
                except Exception as e:
                    pass
                    
        finally:
            # Clean up
            if 'sox_process' in locals():
                sox_process.terminate()
            
            if 'temp_filename' in locals() and os.path.exists(temp_filename):
                os.unlink(temp_filename)
            
            ggwave.free(instance)
        
        return list(received_messages)
    
    return listen_for_encoded_messages(duration, compressor)

# Example usage
if __name__ == "__main__":
    try:
        # Check if we already have a trained compressor
        if os.path.exists("deep_compressor.pkl"):
            print("Loading existing compressor...")
            compressor = TextCompressor("deep_compressor.pkl")
        else:
            # Create the corpus and train a deep compressor
            print("Creating corpus and training compressor...")
            corpus_path = create_deep_corpus()
            compressor = train_deep_compressor(corpus_path)
        
        # Example long message to send
        example_message = """In an age where technology evolves at a breakneck pace, the integration of artificial intelligence into daily life has become not just a possibility, but an inevitability that touches nearly every industry and facet of human interaction, transforming the very way people live, work, learn, and relate to one another."""
        
        # Use the sender with:
        # 1. Higher protocol_id (6) for longer message support
        # 2. Smaller max_message_size (200) to ensure chunks are small enough
        send_encoded_message(example_message, compressor, protocol_id=6, max_bytes=125)
        
        # To listen for messages, uncomment:
        # listen_for_deep_encoded_messages(duration=60, protocol_id=6)
        
    except Exception as e:
        print(f"Error: {e}")

Loading existing compressor...
Message split into 4 chunks based on byte size limit of 125 bytes

Sending chunk 1/4 (48 bytes):
Original message: '[1/4] In an age where technology evolves at a breakneck '
Compressed message: '\x00[1/4] \x10\x13\x0bge½ítechnolog\x92evolvesŔa¤reakneck '
Original: 56 bytes, Compressed: 48 bytes
Compression ratio: 1.17x
Transmitting...
Transmission complete

Sending chunk 2/4 (86 bytes):
Original message: '[2/4] pace, the integration of artificial intelligence into daily life has become not just a possibility, '
Compressed message: '\x00[2/4]čacƴ\x01Ɍegration\x19artificialɌelligencȑ\x82daiĆlifƈs®comeî just¦possibilitȆ'
Original: 106 bytes, Compressed: 86 bytes
Compression ratio: 1.23x
Transmitting...
Transmission complete

Sending chunk 3/4 (84 bytes):
Original message: '[3/4] but an inevitability that touches nearly every industry and facet of human interaction, transforming'
Compressed message: '\x00[3/4]ƶ\x13\x95evitabilit\x92\x1f\x1duchūnearĆevery\x95d